# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import json
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Oqtosh,39.9214,65.9253,19.34,48,0,4.31,UZ,1715487148
1,1,Port Alfred,-33.5906,26.8910,13.54,82,4,5.23,ZA,1715487149
2,2,Kerikeri,-35.2268,173.9474,17.17,61,50,1.34,NZ,1715487150
3,3,Mahina,-17.5065,-149.4890,28.87,65,0,1.54,PF,1715487019
4,4,Tofol,5.3248,163.0078,29.88,79,75,6.17,FM,1715487152


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
df = city_data_df[["City","Lat","Lng","Humidity"]]
hum =city_data_df["Humidity"]
city = city_data_df['City']

# Display the map
df = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Lng,df.Lat))

geo_plot =df.hvplot("Lng",
                    "Lat",
                    color = "City",
                    geo = True,
                    tiles = True,
                    size = hum,
                    width = 1200,
                    height = 800)

geo_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_criteria = city_data_df[(city_data_df['Max Temp']<27)& \
                                  (city_data_df['Max Temp']>21)& \
                                  (city_data_df['Wind Speed']<4.5)& \
                                  (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
city_data_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,Galliano,29.4422,-90.2992,22.22,88,0,0.00,US,1715487207
108,108,Newman,37.3138,-121.0208,26.13,54,0,1.79,US,1715487096
153,153,Foz do Iguaçu,-25.5478,-54.5881,23.51,100,0,1.54,BR,1715487275
225,225,Mossamedes,-15.1961,12.1522,22.28,77,0,2.36,AO,1715487339
249,249,Houma,29.5958,-90.7195,22.22,79,0,0.00,US,1715487359
308,308,Atamyrat,37.8357,65.2106,22.75,35,0,2.63,TM,1715487409
316,316,Marsa Alam,25.0757,34.8918,26.27,65,0,4.13,EG,1715487417
328,328,Cockburn Town,21.4612,-71.1419,26.25,84,0,2.48,TC,1715487428
380,380,São João da Barra,-21.6403,-41.0511,24.14,92,0,4.42,BR,1715487476
391,391,Ilhabela,-23.7781,-45.3581,24.59,63,0,1.79,BR,1715487466


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_criteria.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel_Name"]=''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name
71,71,Galliano,29.4422,-90.2992,22.22,88,0,0.00,US,1715487207,
108,108,Newman,37.3138,-121.0208,26.13,54,0,1.79,US,1715487096,
153,153,Foz do Iguaçu,-25.5478,-54.5881,23.51,100,0,1.54,BR,1715487275,
225,225,Mossamedes,-15.1961,12.1522,22.28,77,0,2.36,AO,1715487339,
249,249,Houma,29.5958,-90.7195,22.22,79,0,0.00,US,1715487359,
308,308,Atamyrat,37.8357,65.2106,22.75,35,0,2.63,TM,1715487409,
316,316,Marsa Alam,25.0757,34.8918,26.27,65,0,4.13,EG,1715487417,
328,328,Cockburn Town,21.4612,-71.1419,26.25,84,0,2.48,TC,1715487428,
380,380,São João da Barra,-21.6403,-41.0511,24.14,92,0,4.42,BR,1715487476,
391,391,Ilhabela,-23.7781,-45.3581,24.59,63,0,1.79,BR,1715487466,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# https://api.geoapify.com/v2/places?
# categories=public_transport.subway
# &filter=circle:-0.07071648508463113,51.50848194136378,1000
# &bias=proximity:-0.07071648508463113,51.50848194136378
# &limit=20&apiKey=YOUR_API_KEY

base_url1 = "https://api.geoapify.com/v2/places"
categories = "accommodation.hotel"
lon =  "-155.9522"
lat =	"19.6228"
raduis = 10000
filters = f"circle:{lon},{lat},{raduis}"
bias = f"proximity:{lon},{lat}"
limit = 5


In [7]:
params = {"categories": categories,
          "filters": filters,
          "bias": bias,
          "limit": limit,
          "apiKey":geoapify_key          
}

In [8]:
response2 = requests.get(base_url1, params=params).json()

In [9]:
response2['features'][0]['properties']['name']

'Kona Hotel'

In [10]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
raduis = 10000
filters = f"circle:{lon},{lat},{raduis}"
bias = f"proximity:{lon},{lat}"
limit = 5

params = {"categories": categories,
          "filters": filters,
          "bias": bias,
          "limit": limit,
          "apiKey":geoapify_key          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{raduis}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url1, params=params)
    
    # Convert the API response to JSON format
    name_address = response2 = requests.get(base_url1, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
Newman - nearest hotel: No hotel found
Foz do Iguaçu - nearest hotel: Hotel Viale Tower
Mossamedes - nearest hotel: Nelsal Pensao
Houma - nearest hotel: Courtyard by Marriott Houma
Atamyrat - nearest hotel: hotel housing working
Marsa Alam - nearest hotel: Blue House Hotel
Cockburn Town - nearest hotel: The Salt Raker Inn
São João da Barra - nearest hotel: Pousada Mediterrâneo
Ilhabela - nearest hotel: Vila Kebaya
Bartlesville - nearest hotel: Hotel Phillips
Al Khārijah - nearest hotel: Kharga Hotel
Duba - nearest hotel: No hotel found
Porto Velho - nearest hotel: Hotel Slaviero Essential Porto Velho
Heroica Caborca - nearest hotel: Hotel Posada del Desierto
Dalbandin - nearest hotel: SF Al-dawood Restaurant
Tamanghasset - nearest hotel: فندق الأمان
Ghat - nearest hotel: فندق تاسيلي
Hanting - nearest hotel: 山东高速服务区管理有限公司青银路潍坊分公司高速宾馆


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel_Name,Hotel Name
71,71,Galliano,29.4422,-90.2992,22.22,88,0,0.00,US,1715487207,,Fairfield Inn & Suites by Marriott Cut Off-Gal...
108,108,Newman,37.3138,-121.0208,26.13,54,0,1.79,US,1715487096,,No hotel found
153,153,Foz do Iguaçu,-25.5478,-54.5881,23.51,100,0,1.54,BR,1715487275,,Hotel Viale Tower
225,225,Mossamedes,-15.1961,12.1522,22.28,77,0,2.36,AO,1715487339,,Nelsal Pensao
249,249,Houma,29.5958,-90.7195,22.22,79,0,0.00,US,1715487359,,Courtyard by Marriott Houma
308,308,Atamyrat,37.8357,65.2106,22.75,35,0,2.63,TM,1715487409,,hotel housing working
316,316,Marsa Alam,25.0757,34.8918,26.27,65,0,4.13,EG,1715487417,,Blue House Hotel
328,328,Cockburn Town,21.4612,-71.1419,26.25,84,0,2.48,TC,1715487428,,The Salt Raker Inn
380,380,São João da Barra,-21.6403,-41.0511,24.14,92,0,4.42,BR,1715487476,,Pousada Mediterrâneo
391,391,Ilhabela,-23.7781,-45.3581,24.59,63,0,1.79,BR,1715487466,,Vila Kebaya


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot

df = hotel_df[["City","Lat","Lng","Humidity","Country", "Hotel Name"]]
# hum = hotel_df["Humidity"]
# city = hotel_df['City']

# Display the map
df = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df.Lng,df.Lat))

geo_plot = df.hvplot.points("Lng",
                            "Lat",
                            color = "City",
                            geo = True,
                            tiles = True,
                            size = "Humidity",
                            width = 1200,
                            height = 800,
                            hover_cols = ["Country", "Hotel Name"]
                            )

# Display the map
geo_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)